# Token classification with Ollama

PAGODA platform at LIRIS.



In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

from pydantic import BaseModel, Field
from typing import List
from ollama_instructor import OllamaInstructor

from datasets import load_dataset
import pandas as pd
import numpy as np
from utils import filter_ner_io


In [2]:
dataset = load_dataset("GEODE/GeoEDdA")
dfs = []
for key in dataset.keys():
    dfs.append(pd.DataFrame({'dataset':key, 'text':dataset[key]['text'], 'meta':dataset[key]['meta'], 'tokens':dataset[key]['tokens'], 'spans':dataset[key]['spans']}))
df = pd.concat(dfs, ignore_index=True)

tagset = ['Domain-mark','Head','NC-Person','NC-Spatial','NP-Misc','NP-Person','NP-Spatial','Relation','Latlong', 'ENE-Spatial', 'ENE-Person', 'ENE-Misc']

df['ner_io'] = df.apply(lambda x: filter_ner_io(x, tagset), axis=1)

df_train = df[df['dataset']=='train'].reset_index(drop = True)
df_val = df[df['dataset']=='validation'].reset_index(drop = True)
df_test = df[df['dataset']=='test'].reset_index(drop = True)

examples_set_train = []
for index,row in df_train.iterrows():
    flat_curr=[element for elements in row['ner_io'] for element in elements]
    examples_set_train.append(len(set(flat_curr)))

idx = np.flip(np.argsort(examples_set_train))[:20]

In [21]:
# Data structure
class Token(BaseModel):
    text: str = Field(description="Texte du token appartenant (ex : 'ville', 'France')")
    #label return a list of labels
    labels: List[str] = Field(description="Liste des labels du token (ex : ['O'], ['Domain-mark'], ['NP-Spatial', 'ENE-Spatial'])")
    #label: str = Field(description="Label de l'entité, exclusivement parmi la liste suivante : ['Domain-mark', 'Head', 'NC-Person', 'NC-Spatial', 'NP-Misc', 'NP-Person', 'NP-Spatial', 'Relation','Latlong', 'ENE-Spatial', 'ENE-Person', 'ENE-Misc','O'] ")

class Tokens(BaseModel):
    entities: List[Token] = Field(description="The token contained in the provided context")



In [22]:

examples=f'''Voici quelques exemples d'entrées et de sorties attendues pour la tâche d'identification des entités dans un texte :
EXEMPLE 0:
    INPUT:{' '.join(["('"+token['text']+"' ,"+str(id)+")" for id,token in enumerate(df_train.iloc[idx[1]]['tokens'])])}
    OUTPUT:{[{'label':tag,'text':token['text']} for tag,token in zip(df_train.iloc[idx[1]]['ner_io'],df_train.iloc[idx[1]]['tokens'])]}
---
'''

directives=f'''
Tu es un expert en Traitement Automatique du Langage Naturel. Ta tâche est de classer chaque token pour identifier les entités (entités nommées, entités nominales et entités imbriquées) dans un texte donné. Les textes sont des paragraphes issus d'articles de l'enyclopédie de Diderot et d'Alembert, et les entités sont des tokens qui peuvent être des noms propres, des noms communs, des relations spatiales ou des coordonnées géographiques.
Les types d'entités possibles sont exclusivement : (Domain-mark, Head, NC-Person, NC-Spatial, NP-Misc, NP-Person, NP-Spatial, Relation, Latlong, ENE-Spatial, ENE-Person, ENE-Misc, O) et peuvent être décrits comme suit :
1. Domain-mark : token indiquant le domaine de connaissance (généralement après le head et entre parenthèses). Par exemple : 'Géog., Géog. mod., Géog. anc., Géogr., Géogr. mod., Marine., Hist. nat., Gram., Géogr. anc., Jurisprud., Géog. anc. & mod., Gramm., Geog.'
2. Head : nom de l'entrée ou article encyclopédique au début de la phrase et est presque toujours en majuscules tel que 'Aire, Afrique, Aigle, ILLESCAS, MULHAUSEN, ADDA, SINTRA ou CINTRA, ACHSTEDE, ou AKSTEDE, KEITH, CAÇERES, CARMAGNOLE, AGRIGNON, INSPRUCK'
3. NC-Person : un nom commun qui identifie une personne telle que 'M., roi, S., peuples, l'empereur, son fils, les habitans, prince, peuple, le roi, fils, le P., habitans'
4. NC-Spatial : un nom commun qui identifie une entité spatiale y compris les caractéristiques naturelles telles que 'ville, petite ville, la riviere, la mer, royaume, la province, capitale, la ville, l'île, cette ville, pays, la côte, riviere'
5. NP-Misc : un nom propre identifiant des entités non classées comme spatiales ou personnelles telles que 'l'Eglise, grec, 1707, russien, Glaciale, Noire, romain, la Croix, Russien, Parlement, 1693, Sud, 1614'
6. NP-Person : un nom propre identifiant le nom d'une personne (entités nommées de personnes) telles que 'Ptolomée, Pline, Strabon, Euripide, les Romains, Pierre, Romains, les Anglois, Turcs, Dieu, César, Antonin, les Espagnols'
7. NP-Spatial : un nom propre identifiant le nom d'un lieu (entités nommées spatiales) telles que 'France, Allemagne, Italie, Espagne, Afrique, Asie, Paris, Naples, Angleterre, Rome, Russie, la Chine, l'Amérique méridionale'
8. Relation : relation spatiale telle que 'dans, sur, au, en, entre, près de, se jette dans, proche, par, vers, près du, jusqu'à, à l'orient'.
9. Latlong : coordonnées géographiques telles que 'Long. 31. 58. lat. 40. 55', 'Long. 10. 27. lat. 43. 30', 'Long. selon Harris, 29. 16. 15. lat. 47. 15'.
10. ENE-Spatial : entité nommée spatiale imbriquée, qui est une entité spatiale qui est composé par d'autres entités, par exemple 'la ville de Paris', 'la province de Bretagne', 'ville de France'.
11. ENE-Person : entité nommée personne imbriquée, qui est une entité faisant référence à une personne et est composé par d'autres entités, par exemple 'le czar Pierre', 'roi de Macédoine'.
12. ENE-Misc : entité nommée non spatiale ou personne imbriquée, qui est une entité faisant référence à un nom propre et est composé par d'autres entités, par exemple "l'ordre de S. Jacques", 'la déclaration du 21 Mars 1671'.
10. O : ce token n'appartient à aucune entité.
Chaque token de l'input doit être classé dans une ou plusieurs de ces catégories. Si un token n'appartient à aucune catégorie, il doit être classé comme 'O'.

{examples}
'''



In [23]:
def run_ollama(user_prompt, client, model_name, data_model, max_retries=3):

    response = client.chat_completion(
        format=data_model,
        model=model_name,
        retries=max_retries,
        messages=[
            {
                'role': 'user',
                'content': user_prompt
            }
        ]
    )
    return Tokens(**json.loads(response.message.content))


In [24]:
PAGODA_API_KEY = os.getenv('PAGODA_API_KEY')

client = OllamaInstructor(
    host='https://ollama-ui.pagoda.liris.cnrs.fr/ollama',
    headers={'Authorization': f'Bearer {PAGODA_API_KEY}'}
    )

version = 'llama3_70b'
model_name = 'llama3:70b'
data_model = Tokens

output_path = os.path.join('predictions', 'token_classification_' + version)

if not os.path.exists(output_path):
    os.makedirs(output_path)

input =  f'''INPUT: {' '.join(["('" + token['text'] + "' ," + str(id) + ")" for id, token in enumerate(df_train.iloc[0]['tokens'])])}'''
user_prompt = directives + input

response = run_ollama(user_prompt, client, model_name, Tokens)

print(response)

entities=[Token(text='ILLESCAS', labels=['NP-Spatial']), Token(text=',', labels=['PONCTUATION']), Token(text='(', labels=['PONCTUATION']), Token(text='Géog', labels=['TYPE_ENTITE']), Token(text='.', labels=['PONCTUATION']), Token(text=')', labels=['PONCTUATION']), Token(text='petite', labels=['ADJECTIF']), Token(text='ville', labels=['NOM_COMMUN']), Token(text="d'", labels=['PRÉPOSITION']), Token(text='Espagne', labels=['NP-Spatial']), Token(text=',', labels=['PONCTUATION']), Token(text='dans', labels=['PRÉPOSITION']), Token(text='la', labels=['ARTICLE_DEFINI']), Token(text='nouvelle', labels=['ADJECTIF']), Token(text='Castille', labels=['NP-Spatial']), Token(text=',', labels=['PONCTUATION']), Token(text='à', labels=['PRÉPOSITION']), Token(text='six', labels=['NOMBRE']), Token(text='lieues', labels=['UNITÉ_DE_MESURE']), Token(text='au', labels=['PRÉPOSITION']), Token(text='sud', labels=['DIRECTION']), Token(text='de', labels=['PRÉPOSITION']), Token(text='Madrid', labels=['NP-Spatial'])

In [25]:
for ent in response.entities:
    print(ent.text, ent.labels)

ILLESCAS ['NP-Spatial']
, ['PONCTUATION']
( ['PONCTUATION']
Géog ['TYPE_ENTITE']
. ['PONCTUATION']
) ['PONCTUATION']
petite ['ADJECTIF']
ville ['NOM_COMMUN']
d' ['PRÉPOSITION']
Espagne ['NP-Spatial']
, ['PONCTUATION']
dans ['PRÉPOSITION']
la ['ARTICLE_DEFINI']
nouvelle ['ADJECTIF']
Castille ['NP-Spatial']
, ['PONCTUATION']
à ['PRÉPOSITION']
six ['NOMBRE']
lieues ['UNITÉ_DE_MESURE']
au ['PRÉPOSITION']
sud ['DIRECTION']
de ['PRÉPOSITION']
Madrid ['NP-Spatial']
. ['PONCTUATION']
